In [1]:
#import numpy as np
#import GCRCatalogs
#from astropy.table import Table
#
#from clmm.structures import GalaxyCluster

In [2]:
## fyi this is clmm/structures/cluster.py
#
#import pickle
#import astropy.table
#from astropy.table import Table
#
#class GalaxyCluster():
#    '''
#    Object that contains the galaxy cluster metadata and background galaxy data
#    Attributes
#    ----------
#    id (int): Unique identifier of the galaxy cluster
#    ra (float): Right ascension of galaxy cluster center (in degrees)
#    dec (float): Declination of galaxy cluster center (in degrees)
#    z (float): Redshift of galaxy cluster center (in degrees)
#    richness (int): Number of member galaxies above prescribes luminosity cut
#    '''
#    def __init__(self, id: int=None, 
#                       ra: float=None, dec: float=None,
#                       z: float=None, richness: int=None,
#                       gals: astropy.table.table.Table=Table()
#                ):
#        self.id = id
#        self.ra = ra
#        self.dec = dec
#        self.z = z
#        self.richness = richness
#        self.gals = gals
#
#    def save(self, filename, **kwargs):
#        """Saves GalaxyCluster object to filename using Pickle"""
#        with open(filename, 'wb') as f:
#            pickle.dump(self, f, **kwargs)
#
#    def load(self, filename, **kwargs):
#        """Loads GalaxyCluster object from filename using Pickle"""
#        with open(filename, 'rb') as f:
#            new_cl = pickle.load(f, **kwargs)
#        self.__init__(**(new_cl.__dict__))


In [12]:
# fyi this is clmm/desc/load_from_dc2.py

import os
import numpy as np
import GCRCatalogs
from astropy.table import Table
import random
from clmm.structures import GalaxyCluster

def load_from_dc2(N, catalog, save_dir):
    print('Loading catalog...')
    catalog = GCRCatalogs.load_catalog(catalog)
    
    print('Getting halos...')
    halos = catalog.get_quantities(['galaxy_id', 'halo_mass', 'redshift','ra', 'dec'],
                                   filters=['halo_mass > 1e14','is_central==True'])
    
    print('Total number of halos: %d'%len(halos))
    print('Analyzing and saving data...')
    for index in random.sample(range(len(halos)), N):
        
        print('Working on No.%d halo...'%index)
        
        i = index #np.random.randint(len(halos))

        cl_id = halos['galaxy_id'][i]
        cl_ra = halos['ra'][i]
        cl_dec = halos['dec'][i]
        cl_z = halos['redshift'][i]
        cl_m = halos['halo_mass'][i]

        # get galaxies around cluster
        ra_min, ra_max = cl_ra-0.3, cl_ra+0.3
        dec_min, dec_max = cl_dec-0.3, cl_dec+0.3
        z_min = cl_z + 0.1
        z_max = 1.5

        coord_filters = [
            'ra >= %s'%ra_min,
            'ra < %s'%ra_max,
            'dec >= %s'%dec_min,
            'dec < %s'%dec_max,
        ]
        z_filters = ['redshift >= %s'%z_min,
                     'redshift < %s'%z_max]

        gals = catalog.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2',
                                       'redshift', 'convergence'], filters=(coord_filters + z_filters))

        # calculate reduced shear
        g1 = gals['shear_1']/(1.-gals['convergence'])
        g2 = gals['shear_2']/(1.-gals['convergence'])

        # store the results into an astropy table
        t = Table([gals['galaxy_id'],gals['ra'],gals['dec'],
                   2*g1, 2*g2,
                   gals['redshift'],gals['convergence']],
                  names=('gal_id','gal_ra','gal_dec', 'gal_e1', 'gal_e2', 'gal_z', 'gal_kappa'))


        c = GalaxyCluster(cl_id=cl_id, cl_ra=cl_ra, cl_dec=cl_dec,
                          cl_z=cl_z, cl_richness=None,
                          gal_cat=t)

        print('Saving No.%d...'%index)
        c.save(os.path.join(save_dir, '%s.p'%cl_id))


In [13]:
catalog = 'cosmoDC2_v1.1.4_small'
#save_dir = '/global/homes/m/mho1/scratch/clmm_scratch/cl_from_gcr'
save_dir = './'

load_from_dc2(5, catalog, save_dir)

Loading catalog...
Getting halos...
Total number of halos: 5
Analyzing and saving data...
Working on No.2 halo...
Saving No.2...
Working on No.4 halo...
Saving No.4...
Working on No.0 halo...
Saving No.0...
Working on No.1 halo...
Saving No.1...
Working on No.3 halo...
Saving No.3...


In [4]:
# code to generate mock data TBD!

# https://github.com/LSSTDESC/CLMM/blob/issue/110/re-refactor-clmm-api/examples/generate_mock_data.ipynb

import support.mock_data as mock
from astropy.table import Table
#import clmm
import clmm.structures as structures


config={}
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4
config['cosmo'] = 'WMAP7-ML'
config['ngals'] = 10000
config['mdef'] = '200c'

ideal_data = mock.MockData(config=config)
noisy_data = mock.MockData(config=config)
# galaxies following redshift distribution, redshift error, shape noise
#noisy_data_z = mock.MockData(config=config)

ideal_data.generate()
noisy_data.generate(is_shapenoise=True, shapenoise=0.005)
#noisy_data_z.generate(is_shapenoise=True, is_zdistribution=True, is_zerr=True)

print(ideal_data.catalog[0])
print(noisy_data.catalog[0])

def save_cluster(cluster_id, mock_data):
    cluster = structures.GalaxyCluster()
    cluster.id = cluster_id
    cluster.ra = 0.
    cluster.dec = 0.
    cluster.z = 0.
    cluster.gals = mock_data.catalog
    cluster_filename = "%d.fits"%cluster_id
    cluster.save(cluster_filename)

save_cluster(123, ideal_data)
save_cluster(456, noisy_data)

# Test
cluster_test = structures.GalaxyCluster()
cluster_test.load("123.fits")
print(cluster_test)
print(cluster_test.gals)


ModuleNotFoundError: No module named 'clmm.models'